**@property가 데코레이션하는 메서드를 같은 클래스에 속하는 여러 애트리뷰트로 사용할 수 없다.**

- 서로 무관한 클래스 사이에서 @property 데코레이터를 적용한 메서드를 재사용할 수 없다.

*디스크립터를 사용하라*

- 애트리뷰트 접근을 해석하는 방법 정의

- __get__와 __set__메서드 제공

   - 별도의 준비코드 없이 원하는 점수 검증 동작을 재사용
   

In [ ]:
#Grade의 인스턴스인 클래스 애트리뷰트가 들어있는 Exam 클래스 정의

class Grade:
    def __get__(self, instance, instance_type):
    def __set__(self, instance, value):

   

class Exam:
    #클래스 애트리뷰트
    math_grade = Grade()
    writig_grade = Grade()
    science_grade = Grade()

exam = Exam()
exam.writing_grade = 40

#Exam.__dict__['writing_grade'].__set__(exam, 40) : 프로퍼티 대입 시 다음과 같이 해석

In [ ]:
exam.writing_grade
#해석: Exam.__dict__['writing_grade'].__get__(exam, 40)

위의 문제점은 writing_grade의 클래스가 모든 Exam 인스턴스에 공유된다.



In [7]:
#데이터 누수를 시키지만 위의 문제 해결
#Grade 인스턴스가 단 한 번만 생성하기에 인스턴스별 상태를 딕셔너리에 저장하여 구현 가능

class Grade:
    def __init__(self):
        self._values = {}

    def __get__(self, instance, instance_type):
        if instance is None:
            return self
        return self._values.get(instance,0)

    def __set__(self, instance, value):
        if not (0<=value<=100):
            raise ValueError(
                '점수는 0과 100사이입니다.'
            )
        self._values[instance] = value

In [ ]:
#누수문제 해결
#weakref내장 모듈 사용

from weakref import WeakKeyDictionary

class Grade:
    def __init__(self):
        self._values = WeakKeyDictionary()

    def __get__(self, instance, instance_type):
        ...
    def __set__(self, instance, value)

In [ ]:
class Exam:
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75


## Summary

- @property 메서드의 동작과 검증 기능을 재사용하고 싶다면 디스크립터 클래스를 만들라

- 디스크립터 클래스를 만들 때는 누수를 방지하기 위해서 WeakKeyDictionary사용

- __getattribute__가 디스크립터 프로토콜을 사용해서 애트리뷰트 값을 읽거나 설정하는 방식을 정확히 이해하라